In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('reviews.csv')

In [3]:
df.head()

,title,description,ratings,sentiment,source,date_posted
0,Worst bike dominar 400,Chain sound is very loud after riding 1500km i...,1,neg,bikewale,25-03-2019
1,The fashionable-- bajaj dominar,"<p>Hi, this bajaj dominar is very good. It has...",3,neg,bikewale,25-04-2018
2,Not as shown in advertisement,"I have test driven it,it is not like shown in ...",1,neg,bikewale,25-04-2018
3,Don't buy dominar 400 or bajaj bikes,Bajaj's quality is so bad as compared to other...,1,neg,bikewale,25-12-2018
4,Servicing and maintenance,"I have dominar bike, this is 1 year old bike r...",1,neg,bikewale,25-11-2018


In [4]:
import re
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

        
decontracted(df['title'][1]+' ' +df['description'][1])

'The fashionable-- bajaj dominar <p>Hi, this bajaj dominar is very good. It has great experience to ride. It is nice and comfortable to ride. Great to buy and as far as looks is concern it is very glamorous and shiny. Its servicing and maintenance is also good and handy at places. So overall it is good to buy this bike.And maintenance is also awesome. So ride well and safely and protect your life with helmet and follow traffic rules.</p>'

In [5]:
from tqdm import tqdm
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't", 'dominar','bajaj','explain','bike']

preprocessed_title = []
# tqdm is for printing the status bar
for sentance in tqdm((df['title']+' ' +df['description']).values):
    sent = decontracted(sentance)
    sent = cleanhtml(sent)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
#     print(sent)
    preprocessed_title.append(sent.lower().strip())
preprocessed_title

100%|████████████████████████████████████████████████████████████████████████████| 1369/1369 [00:00<00:00, 5179.08it/s]


['worst 400 chain sound loud riding 1500km lube chain lubricant also yamaha r15 v3 not faced problem v3 like chain noise lots vibrations worst ever 1 80lakh go ktm 390 pay extra 90 thousand much better worst machine',
 'fashionable hi good great experience ride nice comfortable ride great buy far looks concern glamorous shiny servicing maintenance also good handy places overall good buy maintenance also awesome ride well safely protect life helmet follow traffic rules',
 'not shown advertisement test driven not like shown add troll royal enfield not good performer weight less body hard transmission royal enfield much much better one using enfield legacy sale vehicle problem silencer getting holes sometime use no stability not suitable person height handle congested',
 'not buy 400 bikes quality bad compared bikes not proper engineers team service centres terrible ride lots bikes found brakes not good even dual channel abs scrap not work well service partner not even know service chain 

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
no_features = 1000

In [7]:
tfidf_vectorizer  = TfidfVectorizer(max_df=0.95,min_df=2,stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(preprocessed_title)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [8]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(preprocessed_title)
tf_feature_names = tf_vectorizer.get_feature_names()

In [9]:
from sklearn.decomposition import NMF,LatentDirichletAllocation
no_topics = 20

In [11]:
nmf = NMF(n_components=no_topics,random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)


In [12]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                         for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10


In [13]:
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
good looking 400 handling milage buy comfortable comfort pickup speed
Topic 1:
400 engine cc abs 373 price powerful power comes fuel
Topic 2:
riding experience service performance looks money buying value maintenance long
Topic 3:
km hr months highway easily crosses vibration 100 feels overall
Topic 4:
35 rpm maximum 8000 torque 6500 ps mated generates nm
Topic 5:
best price class segment features budget abs india sports lakhs
Topic 6:
great rider comfort sport performance ride market control goo demand
Topic 7:
duke 390 ktm rc better compared 200 bhp compare bt
Topic 8:
new bikes pulsar company 400 launched brand market dominor indian
Topic 9:
ride amazing love buy feel like smooth speed make friend
Topic 10:
mileage comfort kmpl 30 gives 25 35 super 27 bikes
Topic 11:
really loved like looks superb buy recommend using super braking
Topic 12:
look like light colour design black perfect cool head led
Topic 13:
nice buy comfortable looking price racing driving pickup wonderful 

In [14]:
display_topics(lda, tfidf_feature_names, no_top_words)

Topic 0:
320mm compairing beauty concept disappointed check 29 candy avilable arranged
Topic 1:
27kmpl alway controls advanced creates displacement drag causes abs 1st
Topic 2:
competitors 52 effective 55 capacity easier dark boys bother 110km
Topic 3:
collect earlier beam compairing beauty 4000 consumer 29 constant commendable
Topic 4:
crosswinds 30km burning cbz controll comming claim bit contain air
Topic 5:
breath causes drinks cubic customers conclusion confidence 90 engineering dashing
Topic 6:
dukes 55 engien arm ab designs cruising conclusion colors 320mm
Topic 7:
attract conclusion bracket car dec cornering beauty commendable born 44
Topic 8:
creates attract beauty breaks avilable 20k 43 crossed 44 373
Topic 9:
beauty 162 320 burning complaints arranged check candy creates concept
Topic 10:
110 engin complaints 320 34 advise advance engineering 163km decreases
Topic 11:
162 accept adventurous 26kmpl attracting 120kmph department elephant character complaints
Topic 12:
domina d

In [15]:
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [16]:
tfidf

<1369x2884 sparse matrix of type '<class 'numpy.float64'>'
	with 38511 stored elements in Compressed Sparse Row format>

In [18]:
nmf

NMF(alpha=0.1, beta_loss='frobenius', init='nndsvd', l1_ratio=0.5, max_iter=200,
    n_components=20, random_state=1, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)